In [ ]:
!pip install streamlit pyngrok pyjwt watchdog --quiet


In [ ]:
%%writefile database.py
import sqlite3

def init_db():
    conn = sqlite3.connect("users.db")
    cur = conn.cursor()

    cur.execute("""
    CREATE TABLE IF NOT EXISTS users(
        username TEXT,
        email TEXT PRIMARY KEY,
        password TEXT,
        question TEXT,
        answer TEXT
    )
    """)

    conn.commit()
    conn.close()

init_db()


Overwriting database.py


In [ ]:
%%writefile app.py
import streamlit as st
import sqlite3
import jwt
import datetime
import re

SECRET = "textmorph_secret"

# ================== GLOBAL CREATIVE UI ==================
st.markdown("""
<style>

.stApp {
    background: linear-gradient(-45deg,#ff6ec4,#7873f5,#42e695,#ff9a9e);
    background-size: 400% 400%;
    animation: gradientBG 10s ease infinite;
}

@keyframes gradientBG {
0%{background-position:0% 50%;}
50%{background-position:100% 50%;}
100%{background-position:0% 50%;}
}

.block-container {
    animation: slideUp 0.8s ease;
    background: rgba(255,255,255,0.25);
    backdrop-filter: blur(15px);
    border-radius: 20px;
    padding: 30px;
    border: 3px solid transparent;
    border-image: linear-gradient(45deg,#ff00cc,#3333ff,#00ffcc,#ff6600) 1;
    box-shadow: 0 0 25px rgba(0,0,0,0.3);
}

@keyframes slideUp {
from{opacity:0;transform:translateY(40px);}
to{opacity:1;transform:translateY(0);}
}

h1,h2,h3{
text-align:center;
color:#222;
}

</style>
""", unsafe_allow_html=True)

# ================= DATABASE =================
def init_db():
    conn=sqlite3.connect("users.db")
    cur=conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS users(
        username TEXT,
        email TEXT PRIMARY KEY,
        password TEXT,
        question TEXT,
        answer TEXT
    )
    """)
    conn.commit()
    conn.close()

init_db()

def add_user(u,e,p,q,a):
    conn=sqlite3.connect("users.db")
    cur=conn.cursor()
    cur.execute("INSERT INTO users VALUES(?,?,?,?,?)",(u,e,p,q,a))
    conn.commit()
    conn.close()

def get_user(e):
    conn=sqlite3.connect("users.db")
    cur=conn.cursor()
    cur.execute("SELECT * FROM users WHERE email=?",(e,))
    data=cur.fetchone()
    conn.close()
    return data

def update_password(e,p):
    conn=sqlite3.connect("users.db")
    cur=conn.cursor()
    cur.execute("UPDATE users SET password=? WHERE email=?",(p,e))
    conn.commit()
    conn.close()

# ================= JWT =================
def token(e):
    payload = {
        "email": e,
        "exp": datetime.datetime.utcnow() + datetime.timedelta(minutes=30)
    }

    encoded = jwt.encode(payload, SECRET, algorithm="HS256")
    return encoded


def verify(t):
    try:
        decoded = jwt.decode(t, SECRET, algorithms=["HS256"])
        return decoded
    except:
        return None


# ================= VALIDATION =================
def valid_email(e):
    return re.match(r'^[\w\.-]+@[\w\.-]+\.\w{2,}$',e)

def valid_pass(p):
    return p.isalnum() and len(p)>=8

# ================= SESSION =================
if "page" not in st.session_state:
    st.session_state.page="login"

if "jwt" not in st.session_state:
    st.session_state.jwt=None

# ================= SIGNUP =================
def signup():

    st.title("✨ Create Account")

    u=st.text_input("Username")
    e=st.text_input("Email")
    p=st.text_input("Password",type="password")
    cp=st.text_input("Confirm Password",type="password")

    q=st.selectbox("Security Question",[
        "What is your pet name?",
        "What is your mother’s maiden name?",
        "What is your favorite teacher?"
    ])

    a=st.text_input("Security Answer")

    if st.button("Signup"):

        errors=[]

        if not all([u,e,p,cp,a]):
            errors.append("All fields are mandatory")

        if not valid_email(e):
            errors.append("Invalid email format")

        if not valid_pass(p):
            errors.append("Password must be alphanumeric & 8+ characters")

        if p!=cp:
            errors.append("Passwords do not match")

        if get_user(e):
            errors.append("Email already registered")

        if errors:
            for err in errors:
                st.error(err)
        else:
            add_user(u,e,p,q,a)
            st.success("Account created successfully!")
            st.session_state.page="login"
            st.rerun()

    if st.button("Back to Login"):
        st.session_state.page="login"
        st.rerun()

# ================= LOGIN =================
def login():

    st.title("🔐 Login")

    e=st.text_input("Email")
    p=st.text_input("Password",type="password")

    if st.button("Login"):
        user=get_user(e)

        if user and user[2]==p:
            st.session_state.jwt=token(e)
            st.rerun()
        else:
            st.error("Invalid credentials")

    if st.button("Create Account"):
        st.session_state.page="signup"
        st.rerun()

    if st.button("Forgot Password"):
        st.session_state.page="forgot"
        st.rerun()

# ================= FORGOT PASSWORD =================
def forgot():

    st.title("🔑 Reset Password")

    e = st.text_input("Enter Email")

    if st.button("Verify Email"):

        # ✅ Check empty field
        if not e:
            st.warning("Please enter your email")
            return

        # ✅ Check email format
        if not valid_email(e):
            st.error("Please enter a valid email format (example: user@gmail.com)")
            return

        # ✅ Check database
        user = get_user(e)

        if not user:
            st.error("Email does not exist. Please signup first.")
            return

        # ✅ Show security question
        st.success("Email verified!")
        st.write("Security Question:", user[3])

        ans = st.text_input("Answer")
        newp = st.text_input("New Password", type="password")

        if st.button("Reset Password"):

            if ans == user[4] and valid_pass(newp):
                update_password(e, newp)
                st.success("Password updated successfully!")
                st.session_state.page = "login"
                st.rerun()
            else:
                st.error("Wrong answer or invalid password")

    if st.button("Back"):
        st.session_state.page = "login"
        st.rerun()
# ================= DASHBOARD =================
def dashboard():

    payload = verify(st.session_state.jwt)

    if not payload:
        st.session_state.jwt = None
        st.rerun()

    user = get_user(payload["email"])
    username = user[0]

    # --- Custom UI Styling ---
    st.markdown("""
    <style>

    /* Sidebar */
    section[data-testid="stSidebar"] {
        background: #1e1e2f;
        color: white;
    }

    .sidebar-title {
        font-size: 22px;
        font-weight: bold;
        margin-bottom: 10px;
    }

    /* Hero background */
    .hero {
        background-image: url("https://images.unsplash.com/photo-1501785888041-af3ef285b470");
        background-size: cover;
        background-position: center;
        border-radius: 15px;
        padding: 40px;
        color: white;
        box-shadow: 0 0 20px rgba(0,0,0,0.4);
    }

    .hero h1 {
        font-size: 36px;
        margin-bottom: 5px;
    }

    .hero p {
        font-size: 18px;
        margin-bottom: 20px;
    }

    </style>
    """, unsafe_allow_html=True)

    # ---------- SIDEBAR ----------
    with st.sidebar:
        st.markdown('<div class="sidebar-title">🤖 LLM</div>', unsafe_allow_html=True)

        if st.button("➕ New Chat", use_container_width=True):
            st.info("New chat started!")

        st.markdown("### History")
        st.write("• Project analysis")
        st.write("• NLP")

        st.markdown("---")

        if st.button("Logout", use_container_width=True):
            st.session_state.jwt = None
            st.rerun()

    # ---------- MAIN HERO ----------
    st.markdown(f"""
    <div class="hero">
        <h1>Welcome, {username}!</h1>
        <p>How can I help you today?</p>
    </div>
    """, unsafe_allow_html=True)

    # ---------- CHAT INPUT ----------
    col1, col2 = st.columns([8,1])

    with col1:
        msg = st.text_input("Message", placeholder="Hello! Ask anything about LLM...", label_visibility="collapsed")

    with col2:
        send = st.button("Send")

    if send and msg:
        st.success("Demo reply: I received your message!")

# ================= ROUTER =================
if st.session_state.jwt:
    dashboard()
else:
    if st.session_state.page=="signup":
        signup()
    elif st.session_state.page=="forgot":
        forgot()
    else:
        login()


Overwriting app.py


In [ ]:
!pkill ngrok
!pkill streamlit

from pyngrok import ngrok
import time

# Clear previous tunnels
ngrok.kill()

# Add your token again
token = input("Paste ngrok token: ")
ngrok.set_auth_token(token)

# Start Streamlit
!streamlit run app.py &>/dev/null &

# Wait for server
time.sleep(5)

# Open new tunnel
public_url = ngrok.connect(8501)
print("🚀 App running at:", public_url)